In [1]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

In [2]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100, frac=0.2):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}-{frac}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    return df

In [3]:
def get_metrics(df_res, splits=4, K=None):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        df_res["truth"] = df_res["truth"].fillna(1)
        df = df_res[[sp, "truth"]].dropna()
        t = df["truth"].sum() if K is None else K
        df["pred"] = (df[sp].rank(ascending=False, method="min") <= t) * 1
#         df["pred"] = df[sp] >= 0.5

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [4]:
alg_list = ["fraudar", "rsd", "rev2"][:2]
data_list = ["alpha", "otc", "amazon", "epinions"][:1]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.2, 0.4, 0.6, 0.8]

In [7]:
attack_res[("alpha", "fraudar", 400.0, 0.2)].to_csv("al.csv")

In [6]:
attack_res = {
    (d, a, b, f): get_results(data_name=d, alg=a, attack_name="ilp_attack", budget=b, frac=f)
    for d in data_list for a in alg_list
    for b in budget_list for f in frac_list
}
dict_met = {
    (d, a, b, f): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b, f)], splits=4, K=None))
    for d, a, b, f in attack_res
#     for b in budget_list for f in frac_list
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

0         1         2         3
alpha fraudar 100.0 0.2 precision  0.357513  0.408654  0.380682  0.335526
                        recall     0.862500  1.000000  0.848101  0.698630
                        f1         0.505495  0.580205  0.525490  0.453333
                    0.4 precision  0.364103  0.413953  0.380682  0.335526
                        recall     0.845238  1.000000  0.807229  0.662338
...                                     ...       ...       ...       ...
      rsd     400.0 0.6 recall     0.733871  0.744186  0.772358  0.752137
                        f1         0.733871  0.744186  0.772358  0.752137
                    0.8 precision  0.764286  0.765517  0.798561  0.781955
                        recall     0.764286  0.765517  0.798561  0.781955
                        f1         0.764286  0.765517  0.798561  0.781955

[96 rows x 4 columns]

In [8]:
df_met.index = df_met.index.set_names(["data", "algo", "budget", "frac", "metrics"])

In [9]:
pd.concat([df_met.loc[("alpha", "fraudar", slice(None), slice(None))], df_met.loc[("alpha", "fraudar", slice(None), slice(None))].mean(axis=1)], axis=1)

0         1         2         3         0
budget frac metrics                                                    
100.0  0.2  precision  0.357513  0.408654  0.380682  0.335526  0.370594
            recall     0.862500  1.000000  0.848101  0.698630  0.852308
            f1         0.505495  0.580205  0.525490  0.453333  0.516131
       0.4  precision  0.364103  0.413953  0.380682  0.335526  0.373566
            recall     0.845238  1.000000  0.807229  0.662338  0.828701
            f1         0.508961  0.585526  0.517375  0.445415  0.514319
       0.6  precision  0.357513  0.424658  0.380682  0.335526  0.374595
            recall     0.784091  1.000000  0.770115  0.629630  0.795959
            f1         0.491103  0.596154  0.509506  0.437768  0.508633
       0.8  precision  0.367347  0.377660  0.380682  0.335526  0.365304
            recall     0.782609  0.731959  0.736264  0.600000  0.712708
            f1         0.500000  0.498246  0.501873  0.430380  0.482625
200.0  0.2  precision  0.357513  0.377660  0.380682  0.364198  0.370013
            recall     0.821429  0.797753  0.807229  0.766234  0.798161
            f1         0.498195  0.512635  0.517375  0.493724  0.505482
       0.4  precision  0.357513  0.364641  0.380682  0.335526  0.359590
            recall     0.750000  0.680412  0.736264  0.600000  0.691669
            f1         0.484211  0.474820  0.501873  0.430380  0.472821
       0.6  precision  0.357513  0.386598  0.380682  0.335526  0.365080
            recall     0.690000  0.714286  0.676768  0.548387  0.657360
            f1         0.470990  0.501672  0.487273  0.416327  0.469065
       0.8  precision  0.357513  0.398990  0.380682  0.335526  0.368178
            recall     0.638889  0.699115  0.626168  0.504950  0.617281
            f1         0.458472  0.508039  0.473498  0.403162  0.460793
300.0  0.2  precision  0.364103  0.395939  0.380682  0.335526  0.369062
            recall     0.806818  0.838710  0.770115  0.629630  0.761318
            f1         0.501767  0.537931  0.509506  0.437768  0.496743
       0.4  precision  0.357513  0.460526  0.380682  0.356250  0.388743
            recall     0.690000  1.000000  0.676768  0.612903  0.744918
            f1         0.470990  0.630631  0.487273  0.450593  0.509872
       0.6  precision  0.367347  0.383420  0.384181  0.356250  0.372799
            recall     0.642857  0.632479  0.612613  0.542857  0.607701
            f1         0.467532  0.477419  0.472222  0.430189  0.461841
       0.8  precision  0.357513  0.506224  0.380682  0.364198  0.402154
            recall     0.556452  0.945736  0.544715  0.504274  0.637794
            f1         0.435331  0.659459  0.448161  0.422939  0.491473
400.0  0.2  precision  0.383085  0.440909  0.384181  0.335526  0.385925
            recall     0.836957  1.000000  0.747253  0.600000  0.796052
            f1         0.525597  0.611987  0.507463  0.430380  0.518857
       0.4  precision  0.357513  0.386598  0.380682  0.364198  0.372248
            recall     0.638889  0.663717  0.626168  0.584158  0.628233
            f1         0.458472  0.488599  0.473498  0.448669  0.467310
       0.6  precision  0.357513  0.386598  0.380682  0.335526  0.365080
            recall     0.556452  0.581395  0.544715  0.435897  0.529615
            f1         0.435331  0.464396  0.448161  0.379182  0.431768
       0.8  precision  0.357513  0.395939  0.380682  0.364198  0.374583
            recall     0.492857  0.537931  0.482014  0.443609  0.489103
            f1         0.414414  0.456140  0.425397  0.400000  0.423988

In [10]:
df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "frac", "metrics"], columns=["budget"]).loc[("alpha", "fraudar", slice(None), "f1")].droplevel(0, axis=1)

budget,100.0,200.0,300.0,400.0
frac,,,,
0.2,0.516131,0.505482,0.496743,0.518857
0.4,0.514319,0.472821,0.509872,0.467310
0.6,0.508633,0.469065,0.461841,0.431768
0.8,0.482625,0.460793,0.491473,0.423988
